In [1]:
import configparser
import numpy as np
import os
import sys
import tensorflow as tf

In [2]:
# set to latest model version number

def set_model_version_number():
    version_number = []
    global MODEL_VERSION
    global MODEL_PATH

    if os.path.exists(os.path.join(MODEL_SAVE_DIRECTORY,MODEL_NAME)):   
        for entry in os.listdir(os.path.join(MODEL_SAVE_DIRECTORY,MODEL_NAME)):
            version_number.append(entry)       
        MODEL_VERSION = version_number[-1]
        MODEL_PATH = os.path.join(MODEL_SAVE_DIRECTORY, MODEL_NAME, MODEL_VERSION)
        

In [3]:

config = configparser.ConfigParser()
config.read('config/main.conf')

DATASET = 1
MODEL_VERSION =  "0001"
DOWNLOAD_GOOGLE_LM = False

if DATASET == 1:
    set_dataset = "imdb"
if DATASET == 2:
    set_dataset = "s140"

DATASET_URL = (config[set_dataset]['DATASET_URL'])

DATASET_FOLDER = config[set_dataset]['DATASET_FOLDER']
DATASET_TAR_FILE_NAME = config[set_dataset]['DATASET_TAR_FILE_NAME']
DATASET_NAME = config[set_dataset]['DATASET_NAME']

MODEL_NAME = config[set_dataset]['MODEL_NAME']

CLEAN_DATA_FILE = os.path.join(DATASET_FOLDER,"normalized_dataset.csv")
TAR_FILE_PATH = os.path.join(DATASET_FOLDER,DATASET_TAR_FILE_NAME)
DATA_SET_LOCATION = os.path.join(DATASET_FOLDER,DATASET_NAME)

MODEL_SAVE_DIRECTORY = config[set_dataset]['MODEL_SAVE_DIRECTORY']
# Create the model save directory
if not os.path.exists(MODEL_SAVE_DIRECTORY):
    os.makedirs(MODEL_SAVE_DIRECTORY)
    
IMAGE_SAVE_FOLDER = config[set_dataset]['IMAGE_SAVE_FOLDER']
    
GLOVE_EMBEDDINGS = config[set_dataset]['GLOVE_EMBEDDINGS']
COUNTER_FITTED_VECTORS = config[set_dataset]['COUNTER_FITTED_VECTORS']

GLOVE_EMBEDDINGS_MATRIX = config[set_dataset]['GLOVE_EMBEDDINGS_MATRIX']
COUNTER_FITTED_EMBEDDINGS_MATRIX = config[set_dataset]['COUNTER_FITTED_EMBEDDINGS_MATRIX']

LM_URLS = config[set_dataset]['LM_URLS']
LM_DIRECTORY = config[set_dataset]['LM_DIRECTORY']

####### files required to reconstruct the final trained model ##############################
MODEL_PATH = os.path.join(MODEL_SAVE_DIRECTORY, MODEL_NAME, MODEL_VERSION)

set_model_version_number()

ASSESTS_FOLDER = os.path.join(MODEL_PATH,"assets")
MODEL_ASSETS_VOCABULARY_FILE = os.path.join(ASSESTS_FOLDER,"vocab")
MODEL_ASSETS_EMBEDDINGS_FILE = os.path.join(ASSESTS_FOLDER,"imdb_glove_embeddings_matrix")
MODEL_ASSETS_COUNTER_EMBEDDINGS_FILE = os.path.join(ASSESTS_FOLDER,"counter_embeddings_matrix")
MODEL_ASSETS_DISTANCE_MATRIX = os.path.join(ASSESTS_FOLDER,"distance_matrix.npy")
MODEL_ASSETS_SAVE_BEST_WEIGHTS = os.path.join(ASSESTS_FOLDER, "cp.ckpt")
MODEL_TRAINING_HISORTY_FILE = os.path.join(ASSESTS_FOLDER, "training_history.csv")



### load our pre trained sentiment model

In [4]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization # in Tensorflow 2.1 and above
import pickle 

MAX_VOCABULARY_SIZE = 50000
DIMENSION = 300
LEARNING_RATE = 1e-4


from manny_modules import tf_normalize_data as tfnd
from manny_modules import return_model as rmodel

saved_vocab = pickle.load(open(MODEL_ASSETS_VOCABULARY_FILE, 'rb'))
saved_word_index = dict(zip(saved_vocab, range(len(saved_vocab))))

saved_embeddings_matric = pickle.load(open(MODEL_ASSETS_EMBEDDINGS_FILE, 'rb'))


vectorizer_layer = TextVectorization(
    standardize=tfnd.normlize_data, 
    max_tokens=MAX_VOCABULARY_SIZE, 
    output_mode='int',
    output_sequence_length=300)

# build vocabulary, will also run the normalize_data() 
vectorizer_layer.set_vocabulary(saved_vocab)


saved_model = rmodel.create_model(vectorizer_layer,
                                  saved_embeddings_matric,
                                  saved_vocab,
                                  dimension=DIMENSION, 
                                  lrate=LEARNING_RATE)

# load the weights
saved_model.load_weights(MODEL_ASSETS_SAVE_BEST_WEIGHTS) # loads best weights saved during training

### Test model - check predictions for unseen data

In [5]:
# check negative review
p_2 = [["Seriously, don't bother if you're over 12. This looks like a kids show designed purely to sell merchandise, theme park rides, etc. No logic, holes all over the shop, no characters motivation and really crap acting to top it off... just rubbish, really"]]
prob_positive = saved_model.predict(p_2)

print("Positive confidence: ",prob_positive, " Negative confidence: ", (1 - prob_positive ))


# check positive review
p = [["It's one thing to bring back elements, characters, settings and stories, and to flash them in front of the audience to cash in on the nostalgia and/or recognisable memorabilia but without using it to further the plot and other to do exactly the opposite. It was about time that Star Wars directives understood that it is too unique a product to be lend to corporate filmmakers. Star Wars needs to be understood and its uniqueness has to be acknowledged in order to make the new stories feel like they belong. This may sound too obvious but if you ever wondered why the new SW movies are so controversial this may be the reason.Like with 'Spider-Man: Into the Spider-verse (2018)' and their comicbook-industry experts participation, the creators behind The Mandalorian were experts of the industry, connoisseurs of the Star Wars Universe and even long time fans. So they were able to not only recapture the aesthetic of the grimy, battered Star Wars but also build upon it taking the most 'subtle' things into account. Things like the predominancy of puppets and practical effects over CGI, settings you can feel and touch over green screens and the abundancy of not only known elements previously seen in Star Wars, but a whole batch of new creatures, designs and overall plot elements that felt like they belong to this universe and had always been there. Exceeding expectations are not only the visual aspects but the narrative too. It might be too late for some story elements now, but it is of great importance that from now on you try to watch the unraveling of the story unspoiled. I was lucky to have seen the premiere of the show before the 'memefication' of a certain 'element' that went viral and became one of the biggest highlights of the show. But for me I saw the reveal of this element unspoiled and I was pleasantly shocked, a memory I'll always carry with me. The ability of these creators to generate such shock value and deep moments it's often baffling to me. This is proof that the creators behind the narrative are fully aware of the complexities of the universe they are tampering with and like an experienced surgeon, they are able to tweak, traverse and call back any Star Wars element as they please and with astonishing results."]]
prob_positive = saved_model.predict(p)

print("Positive confidence: ",prob_positive, " Negative confidence: ", (1 - prob_positive ))

Positive confidence:  [[0.01855881]]  Negative confidence:  [[0.9814412]]
Positive confidence:  [[0.9553545]]  Negative confidence:  [[0.04464549]]


### load the distance matrix from disk (load this before running below tests)
- This is a large file (~20GB), so will take time to load

In [6]:

distance_matrix = np.load(MODEL_ASSETS_DISTANCE_MATRIX)


### test the distance matrix

In [7]:
target_word =  saved_word_index['scotland']

In [8]:
from manny_modules import nearest_neighbour as nn

nearest_neighbour, distance_to_neighbour = nn.closest_neighbours(target_word, distance_matrix, number_of_words_to_return=5, max_distance=None)

In [9]:
closest_word = [saved_vocab[x] for x in nearest_neighbour]

print("Words closest to `%s` are `%s` " % (saved_vocab[target_word], closest_word))

Words closest to `scotland` are `['scot', 'scots', 'scottish', 'scotsman', 'scotch']` 


# Genetic Attack

### load dataset

In [10]:
import pandas as pd

dtypes = {'sentiment': 'int', 'text': 'str'}
data_frame = pd.read_csv(CLEAN_DATA_FILE,dtype=dtypes)


### load the saved sample dataset - use the same dataset for both BERT and distance matrix GA attacks

In [67]:
import pandas as pd
from datetime import date

dtypes = {'sentiment': 'int', 'text': 'str'}

# if True then load saved sample dataset, else create a sample dataset
SAVED_DATASET = True
SAMPLE_SIZE = 1000

def load_sample_dataset(): 
    global SAMPLE_SIZE
    if SAVED_DATASET:
        # load the same data-sample used in original attack
        data_sample = pd.read_csv('imdb_dataset/sample_dataset.csv',dtype=dtypes).dropna()
        SAMPLE_SIZE = len(data_sample)
    else:
        #generate new sample from original data set
        data_frame = pd.read_csv(CLEAN_DATA_FILE,dtype=dtypes)
        data_sample = data_frame.sample(n = SAMPLE_SIZE).dropna()
        SAMPLE_SIZE = len(data_sample)
    return data_sample


data_sample = load_sample_dataset()
# save sample dataset with date stamp
data_sample.to_csv('imdb_dataset/sample_dataset_'+ str(date.today()) + '_.csv', index = False)

# show the first 5 randonly selected data items
data_sample.head()


,sentiment,text
0,1,playwright sidney bruhl michael caine has had ...
1,0,yikes did this movie blow the characters were...
2,0,this would have to be one of the worst if not...
3,1,a brilliant sherlock holmes adventure starring...
4,1,first i must say that i don't speak spanish a...


In [68]:
print("Number of data items: ",len(data_sample))

Number of data items:  1006


### add a new column ```probs``` to our sample dataframe to store probability of text being positove (default values = 0)

In [69]:
data_sample['probs'] = 0
data_sample['probs'] = data_sample['probs'].astype(float) # has to be of type float, to store probability values

data_sample = data_sample.reset_index(drop=True) # reindex so we start from 0 in the sample data set
data_sample.head()

,sentiment,text,probs
0,1,playwright sidney bruhl michael caine has had ...,0.0
1,0,yikes did this movie blow the characters were...,0.0
2,0,this would have to be one of the worst if not...,0.0
3,1,a brilliant sherlock holmes adventure starring...,0.0
4,1,first i must say that i don't speak spanish a...,0.0


### now run each one against model and store probabilities

In [70]:
for i in data_sample.index:
    p = saved_model.predict([data_sample.iloc[i]['text']])
    data_sample.at[i,'probs']= p

data_sample.head()

,sentiment,text,probs
0,1,playwright sidney bruhl michael caine has had ...,0.978781
1,0,yikes did this movie blow the characters were...,0.007015
2,0,this would have to be one of the worst if not...,0.016789
3,1,a brilliant sherlock holmes adventure starring...,0.999339
4,1,first i must say that i don't speak spanish a...,0.906772


### add new columns to hold results after genetic attack

In [71]:
# copy current sentiment values to new columns, we can then later go through and compare any values that have been changed during the GA attack
data_sample['ga_sentiment'] = data_sample['sentiment']

# create new ga_text to hold perturbed text
data_sample['ga_text'] = ""

# create ga_probs to hold new probability values after GA Attack, fill with current values
data_sample['ga_probs'] = data_sample['probs']

# create ga_num_changes to hold the number of words changed
data_sample['ga_num_changes'] = 0
data_sample['ga_num_changes'] = data_sample['ga_num_changes'].astype(int)

# create ga_lev_ratio to hold the Levenshtein ratio
data_sample['ga_lev_ratio'] = 0.0
data_sample['ga_lev_ratio'] = data_sample['ga_lev_ratio'].astype(float)

# add field to indicate if sentiment was flipped on review text
data_sample['ga_flipped_sentiment'] = 'N'
data_sample['ga_flipped_sentiment'] = data_sample['ga_flipped_sentiment'].astype(str)


# percentage of words changed in sentence
data_sample['ga_percent_change'] = 0.0
data_sample['ga_percent_change'] = data_sample['ga_percent_change'].astype(float)



data_sample.head()

,sentiment,text,probs,ga_sentiment,ga_text,ga_probs,ga_num_changes,ga_lev_ratio,ga_flipped_sentiment,ga_percent_change
0,1,playwright sidney bruhl michael caine has had ...,0.978781,1,,0.978781,0,0.0,N,0.0
1,0,yikes did this movie blow the characters were...,0.007015,0,,0.007015,0,0.0,N,0.0
2,0,this would have to be one of the worst if not...,0.016789,0,,0.016789,0,0.0,N,0.0
3,1,a brilliant sherlock holmes adventure starring...,0.999339,1,,0.999339,0,0.0,N,0.0
4,1,first i must say that i don't speak spanish a...,0.906772,1,,0.906772,0,0.0,N,0.0


## check the predictions are correct, if not then drop row from data set
### we only want to keep correctly classified data items

In [72]:
drop_indexes = []

for i in data_sample.index:
    if data_sample.iloc[i]['sentiment'] == 1 and data_sample.iloc[i]['probs'] > 0.5:
        continue
    if data_sample.iloc[i]['sentiment'] == 0 and data_sample.iloc[i]['probs'] <= 0.5:
        continue
    else:
        drop_indexes.append(i)
    

In [73]:
data_sample = data_sample.drop(drop_indexes)
data_sample = data_sample.reset_index(drop=True) # reindex dataframe to start from 0

# check how many rows we dropped due to incorrect classification
print("Number of data items dropped from sample: ",(SAMPLE_SIZE - len(data_sample)))
print("Number of data items kept in sample: ",(len(data_sample)))

Number of data items dropped from sample:  0
Number of data items kept in sample:  1006


### GA functions 

In [74]:
from random import randrange
import nltk
from nltk.corpus import stopwords
from pytorch_pretrained_bert import BertTokenizer,BertForMaskedLM
import torch
from transformers import pipeline
import math

stop_words = set(stopwords.words('english')) 

# https://www.scribendi.ai/can-we-use-bert-as-a-language-model-to-assign-score-of-a-sentence/
bertMaskedLM = BertForMaskedLM.from_pretrained('bert-base-uncased')
bertMaskedLM.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#TODO
# https://huggingface.co/bert-base-cased?text=I+%5BMASK%5D+salad+for+lunch
#https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270
# create pipeline to process masked words
mask_word_pipeline = pipeline('fill-mask', model='bert-base-uncased')  

# also re-try using bert-large-uncased
# mask_word_pipeline = pipeline('fill-mask', model='bert-large-uncased') 


def return_masked_words(p, masked_sentence):
    '''return list of possible masked words'''
    return p(masked_sentence)

def get_score(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions=bertMaskedLM(tensor_input)
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(predictions.squeeze(),tensor_input.squeeze()).data
    return math.exp(loss)


def prediction_probability(model, text_review):
    '''return probability of this string being a positive sentiment'''
    return model.predict([text_review])


def crossover(parent_one, parent_two):
    p_one = parent_one.split()
    p_two = parent_two.split()
    
    new_offspring = p_one.copy()
    text_len = min(len(new_offspring), len(p_two))
    # use random univform distribution to select which words to replace 
    # when creating the new offspring for our two parent strings
    for i in range(text_len):
        if np.random.uniform() < 0.5:
            new_offspring[i] = p_two[i]
    return ' '.join(new_offspring)
    
    
def mutation(model, text_review, current_prediction, target_label, max_perturbations, max_neighbours, saved_vocab):
    '''returns the string after swapping max_perturbations nearest neighbours
    of each string'''
    
    # keep track of list index of the which word we have already changed
    selected_index = []
    
    #split string so we can iterate over each word
    t_split = text_review.split()
    
    # select a random index value
    indx = randrange(2, len(t_split) - 3)
    
    for i in range(max_perturbations):
        # get a random index number for word list
        # start at 3rd word and upto 3rd last word index
        indx = randrange(2,len(t_split) - 3)
        
        found_in_vocab = False
        # skip over all stop words and any indexes we have already selected
        while t_split[indx] in stop_words or indx in selected_index  or not found_in_vocab:     
            
            indx = randrange(2,len(t_split) - 3)
            # if word is not found in vocabulary, skip it and try next word
            try:
                target_word = saved_word_index[t_split[indx]]
                found_in_vocab = True
            except KeyError:
                found_in_vocab = False
        
        # now we have a word that is not a stop word and has not already been selected
        selected_index.append(indx) # add to our list
        
        
        ### FITNESS TEST #########
        # we want to now get a list of the closest max_neighbours synonyms using BERT - 
        # we use the two words before and 2 after so we have context for chosen word
        # and then use BERT to return a list of possible substitutions for our masked word
        join_masked = [t_split[indx  - 2],t_split[indx  - 1],'[MASK]',t_split[indx  + 1], t_split[indx  + 2]]
        masked_word_string = ' '.join(join_masked) 
        
        # now we need to pass this string to BERT and get a list of possible substitutions back
        possible_substitutions = return_masked_words(mask_word_pipeline, masked_word_string)
        
        # if one of the words returned is the same as the original word - we want to remove it from our list
        # also remove any non alpha returns
        closest_word = []
        for i in range(len(possible_substitutions)):
            if possible_substitutions[i]['token_str'] == t_split[indx] or not possible_substitutions[i]['token_str'].isalpha():
                continue
            else:
                closest_word.append(possible_substitutions[i]['token_str'])
            
        
        original_word = t_split[indx]
        word_prob_dict = dict()
        for w in closest_word:
            if not w: # if we have an empty string then do nothing, we don't want to remove a word from the string
                continue
            t_split[indx] = w
            word_prob_dict[w] = model.predict([' '.join(t_split)])
            
        # didn't find any suitable words
        if len(word_prob_dict) == 0:
            continue
        
        # the word we decided to substitute is based on the probability returned by the model
        # if we have target_label == 1 then we are trying to go from negative to positive sentiment
        # therefore we want to keep the highest probability returned
        # NB the probability returned by our model is the probability that the review is positive, higher values == more positive sentiment
        if target_label == 1:
            sub_word = max(word_prob_dict, key=word_prob_dict.get)
            t_split[indx] = sub_word
            
        # if our target_label == 0 i.e. negative, then we are going from positive to negative
        # hence we want to keep only the lowest value
        else:
            sub_word = min(word_prob_dict, key=word_prob_dict.get)
            t_split[indx] = sub_word
        
        ### END FITNESS TEST ##########
        
        # add selected index to selected_index list
        selected_index.append(indx)

    return ' '.join(t_split)
    

def generate_population(model, text_review, population_size, current_prediction, target_label, max_perturbations, max_neighbours, saved_vocab):
    
    '''return list of strings of size population_size'''
    
    population = []
    
    for i in range(population_size):
        t = mutation(model, text_review, current_prediction, target_label, max_perturbations, max_neighbours, saved_vocab)
        population.append(t)
    return population



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [81]:
# quick script to recover data sample file after jupyter notebook crash

data_sample.head()
data_sample_BERT.head()

for i in range(len(data_sample_BERT)):
    data_sample.at[i, "sentiment"] = data_sample_BERT.at[i,'sentiment']
    data_sample.at[i, "text"] = data_sample_BERT.at[i,'text']
    data_sample.at[i, "probs"] = data_sample_BERT.at[i,'probs']
    data_sample.at[i, "ga_sentiment"] = data_sample_BERT.at[i,'ga_sentiment']
    data_sample.at[i, "ga_text"] = data_sample_BERT.at[i,'ga_text']
    data_sample.at[i, "ga_probs"] = data_sample_BERT.at[i,'ga_probs']
    data_sample.at[i, "ga_num_changes"] = data_sample_BERT.at[i,'ga_num_changes']
    data_sample.at[i, "ga_lev_ratio"] = data_sample_BERT.at[i,'ga_lev_ratio']
    data_sample.at[i, "ga_flipped_sentiment"] = data_sample_BERT.at[i,'ga_flipped_sentiment']
    data_sample.at[i, "ga_percent_change"] = data_sample_BERT.at[i,'ga_percent_change']
    
    


In [ ]:
# print(data_sample.at[(len(data_sample_BERT) - 5),'probs'])
# print(data_sample.at[(len(data_sample_BERT) - 5),'ga_probs'])
# len(data_sample_BERT) - 5

In [165]:
import Levenshtein as lev
import time

POPULATION_SIZE = 40 # max population size to create 
MAXIMUM_ITERATIONS = 300 # stopping condition for loop if we do not find an optimal solution increased to 300 for BERT attack
MAX_PERTURBATIONS = 5 # maximum number of changes to make for each population member increase from 5 to 6 for BERT
MAX_NEIGHBOURS = 4 # maximum number of neighbouring words to return and check against


def population_probs_df(population_list, model):
    # dataframe to store population text and probabilities
    population_probs = pd.DataFrame(columns=['ga_text','ga_probs'])
    

    # make sure columns have the correct types
    population_probs['ga_text'] = population_probs['ga_text'].astype(str)
    population_probs['ga_probs'] = population_probs['ga_probs'].astype(float)
    
    for i in range(len(population_list)):
        new_row = {'ga_text':population_list[i], 'ga_probs':model.predict([population_list[i]])}
        #append row to the dataframe
        population_probs = population_probs.append(new_row, ignore_index=True)
    
    # sort the array by probabilities column, i.e column 2 
    return  population_probs.sort_values('ga_probs') # return sorted df, sorted by probability lowest to highest
    

def found_solution(target_label, population_df):
    
    if target_label == 1:
        if population_df.iloc[-1]['ga_probs'] > 0.5:
            return True
    
    if target_label == 0:
        if population_df.iloc[0]['ga_probs'] <= 0.5:
            return True 
    return False


def number_of_changes_made(review_before, review_after):
    word_count = 0
    r_before = review_before.split()
    r_after = review_after.split()
    for i in range(len(r_before)):
        if r_before[i] == r_after[i]:
            pass
        else:
            word_count += 1
    return word_count


def GA_Attack_BERT(data_sample):
    data_sample_len = len(data_sample)

    for i in range(data_sample_len):
    
        print("####### Data Item: ",i+1," #######")
        target_label = 0 if data_sample.iloc[i]['sentiment'] == 1 else 1
        current_prediction = data_sample.iloc[i]['probs']
        
        # calculate how long it takes us to process each data item
        t0 = time.time()
        
        p = generate_population(saved_model, data_sample.iloc[i]['text'], POPULATION_SIZE, current_prediction, target_label, MAX_PERTURBATIONS, MAX_NEIGHBOURS, saved_vocab)
    
        population_dataframe = population_probs_df(p, saved_model)

    
        ## GA Attack START
        # need to run through and do crossover and mutation, recheck the label and if it has flipped then we stop, other wise keep going
        # also on each iteration keep updating the ga_text and ga_prob and if label has changed update the ga_sentiment column and stop
        for j in range(MAXIMUM_ITERATIONS):
        
        
            # first check if we have found a solution, if yes then we are done so save results and break
            # and move onto next
            if found_solution(target_label, population_dataframe):
                #save solution
                if target_label == 1:   
                    data_sample.at[i, "ga_text"] = population_dataframe.iloc[-1]['ga_text']
                    data_sample.at[i, "ga_probs"] = population_dataframe.iloc[-1]['ga_probs']
                    data_sample.at[i, "ga_sentiment"] = 1
                    break
                if target_label == 0:
                    data_sample.at[i, "ga_text"] = population_dataframe.iloc[0]['ga_text']
                    data_sample.at[i, "ga_probs"] = population_dataframe.iloc[0]['ga_probs']
                    data_sample.at[i, "ga_sentiment"] = 0
                    break
        
            # select the two best parents from the population
            if target_label == 1:
                parent_one = population_dataframe.iloc[-1]['ga_text']
                parent_two = population_dataframe.iloc[-2]['ga_text']
            else: 
                parent_one = population_dataframe.iloc[0]['ga_text']
                parent_two = population_dataframe.iloc[1]['ga_text']
        
            # save progress so far
            if target_label == 1 and (current_prediction < population_dataframe.iloc[-1]['ga_probs']):
                data_sample.at[i, "ga_text"] = population_dataframe.iloc[-1]['ga_text']
                data_sample.at[i, "ga_probs"] = population_dataframe.iloc[-1]['ga_probs']
            
            if target_label == 0 and (current_prediction > population_dataframe.iloc[0]['ga_probs']):
                data_sample.at[i, "ga_text"]  = population_dataframe.iloc[0]['ga_text']
                data_sample.at[i, "ga_probs"] = population_dataframe.iloc[0]['ga_probs']
        
            # we didn't find a solution yet, so we do crossover and generate a new population of possible solutions 
            t = crossover(parent_one, parent_two)
            p = generate_population(saved_model, t, POPULATION_SIZE, current_prediction, target_label, MAX_PERTURBATIONS, MAX_NEIGHBOURS, saved_vocab)
            population_dataframe = population_probs_df(p, saved_model)
        
        num_words_changes = number_of_changes_made(data_sample.at[i, "text"], data_sample.at[i, "ga_text"])
        total_words_in_review = len(data_sample.at[i, "text"].split())
        
        print("\tElapsed time: ", round(time.time() - t0), " seconds")
        print("\tNumber of words in review: ",total_words_in_review )
        print("\tNumber of words swapped: ", num_words_changes)
        print("\tPercentage modified: ", round((num_words_changes / total_words_in_review ),2) * 100,"%")
        print("\tProb. before and after: ", data_sample.at[i, "probs"]," : ", data_sample.at[i, "ga_probs"],"\n")
        
    return data_sample

 


### start the GA Attack

In [167]:
data_sample = GA_Attack_BERT(data_sample)

### save the results after running GA Attack

In [ ]:

def save_ga_attack_results(data_sample):
    # save the final set of results
    data_sample.to_csv('imdb_dataset/ga_results_BERT.csv', index = False)

    # load saved file, so we can remove any results that were not processed
    dtypes = {'sentiment': 'int', 
              'text': 'str', 
              'probs': 'float', 
              'ga_sentiment': 'int', 
              'ga_text': 'str',
              'ga_probs': 'float', 
              'ga_num_changes': 'int', 
              'ga_lev_ratio': 'float', 
              'ga_flipped_sentiment': 'str',
             'ga_percent_change': 'float'}

    data_sample = pd.read_csv('imdb_dataset/ga_results_BERT.csv', dtype=dtypes)

    # drop any rows with nan value - i.e. data items not processed due to Jupyter notebook crash
    # so we don't have to re-run the whole GA Attack again
    data_sample = data_sample.dropna()

    # save the final set of results
    data_sample.to_csv('imdb_dataset/ga_results_BERT.csv', index = False)


In [149]:
data_sample.head()

,sentiment,text,probs,ga_sentiment,ga_text,ga_probs,ga_num_changes,ga_lev_ratio,ga_flipped_sentiment,ga_percent_change
0,0,michael caine usually appears in either very g...,0.000544,1,michael caine first appears in either very or ...,0.585462,40,0.896853,Y,0.13
1,0,this is a poor poor movie full of clichés u...,0.001128,1,this is a good family movie because of the unr...,0.775919,10,0.935867,Y,0.07
2,1,utterly tactical strange watch for the kinky ...,0.675391,0,utterly tactical strange watch for the kinky e...,0.437926,4,0.984972,Y,0.01
3,0,do we really need any more narcissistic garbag...,0.156599,1,do we really need any more narcissistic garbag...,0.585778,5,0.948393,Y,0.05
4,1,capt corelli's mandolin is an old fashioned h...,0.909436,0,capt corelli's name is an old two books on rom...,0.246867,11,0.856757,Y,0.16


### number of test data items processed during GA Attack

In [92]:
print("Number of data items processed in GA Attack: ", len(data_sample))

Number of data items processed in GA Attack:  1006


### calculate Levenshtein ratio and percentage change after GA Attack

In [93]:
import Levenshtein as lev

# calculate Levenshtein ratio for text and ga_text
# text == initial review text, ga_text == review text after GA Attack
# value closer to 1.0 indicates more similarity, i.e. less changes made to original text
for i in range (len(data_sample)):
    data_sample.at[i,'ga_lev_ratio'] = lev.ratio(data_sample.at[i,'text'],data_sample.at[i,'ga_text'])
    num_words_changes = number_of_changes_made(data_sample.at[i, "text"], data_sample.at[i, "ga_text"])
    total_words_in_review = len(data_sample.at[i, "text"].split())
    data_sample.at[i,'ga_percent_change'] = num_words_changes/total_words_in_review

data_sample.head()


,sentiment,text,probs,ga_sentiment,ga_text,ga_probs,ga_num_changes,ga_lev_ratio,ga_flipped_sentiment,ga_percent_change
0,0,michael caine usually appears in either very g...,0.000544,1,michael caine first appears in either very or ...,0.585462,0,0.896853,N,0.126582
1,0,this is a poor poor movie full of clichés u...,0.001128,1,this is a good family movie because of the unr...,0.775919,0,0.935867,N,0.066667
2,1,utterly tactical strange watch for the kinky ...,0.675391,0,utterly tactical strange watch for the kinky e...,0.437926,0,0.984972,N,0.009877
3,0,do we really need any more narcissistic garbag...,0.156599,1,do we really need any more narcissistic garbag...,0.585778,0,0.948393,N,0.054348
4,1,capt corelli's mandolin is an old fashioned h...,0.909436,0,capt corelli's name is an old two books on rom...,0.246867,0,0.856757,N,0.164179


### check for which reviews we successfully flipped the sentiment and set ```ga_flipped_sentiment``` 

In [94]:
flipped_count = 0
failed_flipped_count = 0
for i in range(len(data_sample)):
    if  data_sample.at[i,'sentiment'] != data_sample.at[i,'ga_sentiment'] and (data_sample.at[i,'ga_percent_change'] <= 0.2):
        data_sample.at[i,'ga_flipped_sentiment'] = 'Y'
        flipped_count += 1
    else:
        failed_flipped_count += 1
        data_sample.at[i,'ga_flipped_sentiment'] = 'N'
        
print("Percentage of reviews where sentiment was changed after attack:", round(flipped_count/len(data_sample) * 100, 2),"%", "changed sentiment. i.e.", flipped_count, " out of ",len(data_sample))
print("Percentage of reviews failed to change sentiment: ", round(failed_flipped_count/len(data_sample) * 100, 2), "%","did not change, i.e.", failed_flipped_count, " out of ",len(data_sample))

Percentage of reviews where sentiment was changed after attack: 97.42 % changed sentiment. i.e. 980  out of  1006
Percentage of reviews failed to change sentiment:  2.58 % did not change, i.e. 26  out of  1006


In [112]:
data_sample.head()

,sentiment,text,probs,ga_sentiment,ga_text,ga_probs,ga_num_changes,ga_lev_ratio,ga_flipped_sentiment,ga_percent_change
0,0,michael caine usually appears in either very g...,0.000544,1,michael caine first appears in either very or ...,0.585462,40,0.896853,Y,0.13
1,0,this is a poor poor movie full of clichés u...,0.001128,1,this is a good family movie because of the unr...,0.775919,10,0.935867,Y,0.07
2,1,utterly tactical strange watch for the kinky ...,0.675391,0,utterly tactical strange watch for the kinky e...,0.437926,4,0.984972,Y,0.01
3,0,do we really need any more narcissistic garbag...,0.156599,1,do we really need any more narcissistic garbag...,0.585778,5,0.948393,Y,0.05
4,1,capt corelli's mandolin is an old fashioned h...,0.909436,0,capt corelli's name is an old two books on rom...,0.246867,11,0.856757,Y,0.16


### make sure all reviews are of the same length before and after GA Attack
i.e. make sure we didn't remove any words

In [95]:
len_diff_count = 0
# check to make sure no words were completely removed form reviews
for i in range(len(data_sample)):
    if  len(data_sample.at[i,'text'].split()) != len(data_sample.at[i,'ga_text'].split()):
        len_diff_count += 1
print(len_diff_count)

0


### count how many words were changed for each review, average number of word changed and percentage of change to review

In [96]:
word_total_count = 0
percent_modified_total = 0.0
for i in range(len(data_sample)):
    review_before = data_sample.at[i,'text'].split()
    review_after = data_sample.at[i,'ga_text'].split()
    word_count = 0
    for j in range(len(review_before)):
        if review_before[j] != review_after[j]:
            word_count += 1
    data_sample.at[i, 'ga_num_changes'] = word_count
    percent_modified_total += (word_count / len(review_before))
    data_sample.at[i, 'ga_percent_change'] = round(word_count / len(review_before), 2)
    word_total_count += word_count
            
data_sample.head()

print("Avg. num of words changed changes made: ", (int)(word_total_count / len(data_sample)))
print("Avg. percentage modified: ", (round(percent_modified_total / len(data_sample), 2) * 100),"%")
      

Avg. num of words changed changes made:  15
Avg. percentage modified:  8.0 %


### load the original raw data-file

In [102]:
from manny_modules import normalize_dataset as nd
# load raw dataset i.e. before we normalised it
dtypes = {'sentiment': 'int', 
          'text': 'str' 
         }

raw_data_sample = pd.read_csv('imdb_dataset/raw_data.csv', dtype=dtypes)
rds = raw_data_sample.copy()

# normalise the dataset
normalized_dataset = nd.clean_and_return(rds, 'text')


In [103]:
print(raw_data_sample.head())
print(normalized_dataset.head())

   sentiment                                               text
0          1  This is a delightful movie that is so over-the...
1          1  After, I watched the films... I thought, "Why ...
2          1  This short deals with a severely critical writ...
3          0  Just Cause is one of those films that at first...
4          0  For reasons I cannot begin to fathom, Dr. Lore...
   sentiment                                               text
0          1  this is a delightful movie that is so overthet...
1          1  after  i watched the films  i thought  why the...
2          1  this short deals with a severely critical writ...
3          0  just cause is one of those films that at first...
4          0  for reasons i cannot begin to fathom  dr  lore...


### add to a list all the data-sample indexes that match against the raw datafile

In [137]:
data_sample_BERT = pd.read_csv('imdb_dataset/ga_results_BERT.csv', dtype=dtypes)

index_values_rawdata = []
for i in range(len(data_sample_BERT)):
    for j in range(len(normalized_dataset)): 
        if normalized_dataset.at[j,'text'] == data_sample_BERT.at[i, 'text']:
           # print(raw_data_sample.at[j,'text'],"\n",data_sample.at[i, 'text'],"\n\n")
            index_values_rawdata.append(j)
            break


In [147]:
print(len(index_values_rawdata))
print(list(index_values_rawdata))

In [144]:
print(raw_data_sample.at[35562,'text'])
print("\n")
print(normalized_dataset.at[35562,'text'])
print("\n")
print(data_sample.at[1,'ga_text'])
print("\nBEFORE:\t\t",data_sample.at[1,'probs'])
print("AFTER:\t\t",data_sample.at[1,'ga_probs'])
print("PERCENT CHANGE:\t",round(data_sample.at[1,'ga_percent_change'] * 100), "%")
print("WORDS CHANGED:\t",data_sample.at[1,'ga_num_changes'])


This is a poor, poor movie. Full of clichés, unrealistic moments: punching the air in celebration after putting a fire out, never mind that someone's lost their home and possessions!!, announcing a pregnancy in a bar along with all your mates before telling you in private first, walking on the roof of a burning building for no apparent reason, the stereotypical funerals and strained relationships, the very dodgy, cheesy music at the end, the unrealistic treatment of the girl who was rescued from her apartment, the very unrealistic explosion from that same apartment!! Did they have a couple of oxygen tanks in the attic or something!!? Anyone with an ounce of wit can see that this movie was a joke. It's a pity, because firefighters do an awesome job, and they deserve to have a good movie made about what they do, but not at the expense of common sense.


this is a poor  poor movie  full of clichés  unrealistic moments  punching the air in celebration after putting a fire out  never mind t

In [145]:
raw_split_words = normalized_dataset.at[35562,'text'].split()
data_sample_split_words = data_sample_BERT.at[1,'ga_text'].split()

len_range = len(data_sample_BERT.at[1,'text'].split())

for i in range(len_range):
    if raw_split_words[i] != data_sample_split_words[i]:
        print("BEFORE: ", raw_split_words[i], " AFTER: ",data_sample_split_words[i])


BEFORE:  poor  AFTER:  good
BEFORE:  poor  AFTER:  family
BEFORE:  full  AFTER:  because
BEFORE:  clichés  AFTER:  the
BEFORE:  pregnancy  AFTER:  change
BEFORE:  burning  AFTER:  public
BEFORE:  dodgy  AFTER:  best
BEFORE:  unrealistic  AFTER:  medical
BEFORE:  explosion  AFTER:  results
BEFORE:  made  AFTER:  news


In [146]:
print(raw_split_words)
print("\n")
print(data_sample_split_words)

['this', 'is', 'a', 'poor', 'poor', 'movie', 'full', 'of', 'clichés', 'unrealistic', 'moments', 'punching', 'the', 'air', 'in', 'celebration', 'after', 'putting', 'a', 'fire', 'out', 'never', 'mind', 'that', "someone's", 'lost', 'their', 'home', 'and', 'possessions', 'announcing', 'a', 'pregnancy', 'in', 'a', 'bar', 'along', 'with', 'all', 'your', 'mates', 'before', 'telling', 'you', 'in', 'private', 'first', 'walking', 'on', 'the', 'roof', 'of', 'a', 'burning', 'building', 'for', 'no', 'apparent', 'reason', 'the', 'stereotypical', 'funerals', 'and', 'strained', 'relationships', 'the', 'very', 'dodgy', 'cheesy', 'music', 'at', 'the', 'end', 'the', 'unrealistic', 'treatment', 'of', 'the', 'girl', 'who', 'was', 'rescued', 'from', 'her', 'apartment', 'the', 'very', 'unrealistic', 'explosion', 'from', 'that', 'same', 'apartment', 'did', 'they', 'have', 'a', 'couple', 'of', 'oxygen', 'tanks', 'in', 'the', 'attic', 'or', 'something', 'anyone', 'with', 'an', 'ounce', 'of', 'wit', 'can', 'see'

In [21]:


masked_sentence = "flying [MASK] in the sky"
word_list = return_masked_words(mask_word_pipeline,masked_sentence)

masked_words = []
for i in range(len(word_list)):
    if not word_list[i]['token_str'].isalpha():
        masked_words.append(word_list[i]['token_str'])
masked_words

[]

In [348]:
t = data_sample.at[0,'text'].split()
indx_val = randrange(2,(len(t) - 3))
print(indx_val,len(t))
join_list = [t[indx_val  - 2],t[indx_val  - 1],'[MASK]',t[indx_val  + 1], t[indx_val  + 2] ]
o_join_list = [t[indx_val  - 2],t[indx_val  - 1],t[indx_val],t[indx_val  + 1], t[indx_val  + 2] ]

83 234


In [349]:
masked_word_to_check = ' '.join(join_list )
o_sent = ' '.join(o_join_list )
print(o_sent)
print(masked_word_to_check)

working full force caine is
working full [MASK] caine is


In [350]:
possible_substitutions = return_masked_words(mask_word_pipeline, masked_word_to_check)
print(possible_substitutions)

[{'sequence': '[CLS] working full time caine is [SEP]', 'score': 0.9468444585800171, 'token': 2051, 'token_str': 'time'}, {'sequence': '[CLS] working full on caine is [SEP]', 'score': 0.013376843184232712, 'token': 2006, 'token_str': 'on'}, {'sequence': '[CLS] working full, caine is [SEP]', 'score': 0.003743428271263838, 'token': 1010, 'token_str': ','}, {'sequence': '[CLS] working full circle caine is [SEP]', 'score': 0.0018968889489769936, 'token': 4418, 'token_str': 'circle'}, {'sequence': '[CLS] working full as caine is [SEP]', 'score': 0.0017040419625118375, 'token': 2004, 'token_str': 'as'}]


In [355]:
print(possible_substitutions[0]['sequence'])

[CLS] working full time caine is [SEP]
